[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/12d-nHER54IHl-yDMSoSSqi8ywuUDL1rw?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator, DateFormatter
import seaborn as sns 
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose


In [ ]:
fulldata = pd.read_csv('/content/drive/MyDrive/twitter/Final_Tweets_sentiment.csv')
cleandata=fulldata[["tweet_date","city" ,"country", "latitude","longitude","predicted_class"]]
cleandata.head(5)

,tweet_date,city,country,latitude,longitude,predicted_class
0,2020-01-31T23:55:01+00:00,Colorado,United States,40.014986,-105.270546,1.0
1,2020-01-31T23:37:00+00:00,Colorado,United States,40.014986,-105.270546,1.0
2,2020-01-31T23:36:58+00:00,New York,United States,40.712775,-74.005973,1.0
3,2020-01-31T23:33:01+00:00,Colorado,United States,40.014986,-105.270546,1.0
4,2020-01-31T23:31:51+00:00,Colorado,United States,40.014986,-105.270546,1.0


In [ ]:
cleandata['date'] = pd.to_datetime(cleandata['tweet_date'])
cleandata["Year_month"]=cleandata['date'].dt.strftime('%Y-%m')
ts_df=cleandata[["Year_month", "predicted_class", "country"]]
ts_df.head(10)

<ipython-input-4-d26081f9cd76>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandata['date'] = pd.to_datetime(cleandata['tweet_date'])
<ipython-input-4-d26081f9cd76>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleandata["Year_month"]=cleandata['date'].dt.strftime('%Y-%m')


,Year_month,predicted_class,country
0,2020-01,1.0,United States
1,2020-01,1.0,United States
2,2020-01,1.0,United States
3,2020-01,1.0,United States
4,2020-01,1.0,United States
5,2020-01,1.0,United States
6,2020-01,1.0,United States
7,2020-01,1.0,United States
8,2020-01,0.0,United States
9,2020-01,0.0,Canada


In [ ]:
ts_df['Year_month'] = pd.to_datetime(ts_df['Year_month'])
grouped = ts_df.groupby(['country',"Year_month"])['predicted_class'].agg(total_sentiment_count='count', negative_sentiment_count=lambda x: sum(x == 0))
grouped["depress_score"]=grouped["negative_sentiment_count"]/grouped["total_sentiment_count"]
del grouped['total_sentiment_count']
del grouped['negative_sentiment_count']
grouped.head(15)


<ipython-input-6-0beb05dbce9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts_df['Year_month'] = pd.to_datetime(ts_df['Year_month'])


depress_score
country     Year_month               
Afghanistan 2020-03-01       0.000000
            2020-06-01       0.000000
            2020-07-01       0.333333
            2020-10-01       0.000000
            2020-11-01       1.000000
            2021-02-01       0.000000
            2021-04-01       0.000000
            2021-07-01       0.500000
            2021-09-01       0.333333
            2022-01-01       0.000000
            2022-03-01       1.000000
            2022-04-01       0.000000
            2022-05-01       0.333333
            2023-03-01       0.000000
Albania     2020-05-01       1.000000

In [ ]:
count_by_country = grouped.groupby(level='country').size()
count_by_country.head(5)


country
Afghanistan    14
Albania         4
Algeria        12
Argentina      22
Armenia         3
dtype: int64

In [ ]:

selected_countries = count_by_country[count_by_country >= 30].index
df_filtered = grouped.loc[selected_countries]
df_filtered.head(30)

depress_score
country   Year_month               
Australia 2020-01-01       0.239521
          2020-02-01       0.225962
          2020-03-01       0.252427
          2020-04-01       0.211207
          2020-05-01       0.193694
          2020-06-01       0.158879
          2020-07-01       0.160714
          2020-08-01       0.212291
          2020-09-01       0.278689
          2020-10-01       0.185185
          2020-11-01       0.170370
          2020-12-01       0.236220
          2021-01-01       0.152381
          2021-02-01       0.227941
          2021-03-01       0.213793
          2021-04-01       0.294118
          2021-05-01       0.194444
          2021-06-01       0.279570
          2021-07-01       0.169355
          2021-08-01       0.169935
          2021-09-01       0.250000
          2021-10-01       0.185484
          2021-11-01       0.154639
          2021-12-01       0.158537
          2022-01-01       0.227848
          2022-02-01       0.285714
          2022-03-01       0.183486
          2022-04-01       0.208333
          2022-05-01       0.223140
          2022-06-01       0.259259

In [ ]:
df_filtered.to_csv('/content/drive/MyDrive/twitter/cleaned.csv', index=True)

In [ ]:
groups = df_filtered.groupby(level='country')
for name, group in groups:
    print('Time series analysis for:', name)
    # extract the time series data for this country
    ts = group['depress_score']
    
    # reset index of the time series to a single level index
    ts = ts.reset_index(level=0, drop=True)
    
    # perform seasonal decomposition of the time series
    decomposition = sm.tsa.seasonal_decompose(ts, model='additive', period=12)
    
    # calculate rolling 3-month moving average
    rolling_mean = ts.rolling(window=3).mean()
    
    # plot the time series and rolling average
    fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(6, 6))
    ax[0].set_title(name)
    ax[0].plot(ts, label='Original')
    ax[0].legend()
    ax[1].plot(decomposition.trend, label='Trend')
    ax[1].legend()
    ax[2].plot(decomposition.seasonal, label='Seasonal')
    ax[2].legend()
    ax[3].plot(decomposition.resid, label='Residual')
    ax[3].legend()
    locator = YearLocator()
    formatter = DateFormatter('%Y')
    for i in range(4):
        ax[i].xaxis.set_major_locator(locator)
        ax[i].xaxis.set_major_formatter(formatter)
    plt.tight_layout()
    #plt.show()
    filename = '/content/drive/MyDrive/twitter/Img/' + name + '_decomposition.png'
    plt.savefig(filename, dpi=300)
    plt.close(fig)

Time series analysis for: Australia
Time series analysis for: Austria
Time series analysis for: Bangladesh
Time series analysis for: Belgium
Time series analysis for: Canada
Time series analysis for: China
Time series analysis for: Denmark
Time series analysis for: Finland
Time series analysis for: France
Time series analysis for: Germany
Time series analysis for: Ghana
Time series analysis for: India
Time series analysis for: Indonesia
Time series analysis for: Ireland
Time series analysis for: Italy
Time series analysis for: Japan
Time series analysis for: Kenya
Time series analysis for: Malaysia
Time series analysis for: Netherlands
Time series analysis for: New Zealand
Time series analysis for: Nigeria
Time series analysis for: Pakistan
Time series analysis for: Philippines
Time series analysis for: Portugal
Time series analysis for: Saudi Arabia
Time series analysis for: South Africa
Time series analysis for: Spain
Time series analysis for: Sweden
Time series analysis for: Switzer